<a href="https://colab.research.google.com/github/abayega/Courses-and-Practicals/blob/master/phyla_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Very basic microbiome data classifier

PyTorch Cancer Classification Pipeline
Copy of AIforGenomics_2021_session_2.ipynb

In [14]:
#Import packages

import os
from IPython.display import Image

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, random_split, DataLoader

import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import MNIST, ImageFolder
from torchvision.utils import make_grid, save_image

import numpy as np
import pandas as pd

import matplotlib
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt

%matplotlib inline
matplotlib.rcParams['figure.facecolor'] = '#ffffff'

In [2]:
#Load google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
#Read data file
data_file = F'/content/gdrive/MyDrive/PhD/Coursework/AI in Genomics/Projects/Phyla/phyla_dataset_d3/phyla_dataset_d3.csv'


In [4]:
#Pandas data read
microbe_df = pd.read_csv(data_file)

In [12]:
#Pandas data view
print(microbe_df.iloc[1:4,1178:1184])
print(microbe_df.iloc[1:4,1:2])
#microbe_df.groupby(['col_site','uc_cd']).uc_cd.count()
#microbe_df.groupby(['col_site','stool_biopsy']).stool_biopsy.count()
#microbe_df.col_site.value_counts()
#microbe_df.uc_cd.value_counts()
#microbe_df.stool_biopsy.value_counts()
#microbe_df.shape

  col_site  diagnosis sample_title stool_biopsy  studyID uc_cd
1   OSCCAR          1  1939.100003        stool  GEVERSM    CD
2   OSCCAR          1  1939.100009        stool  GEVERSM    UC
3   OSCCAR          1  1939.100015        stool  GEVERSM    CD
   D_0__Archaea;D_1__Euryarchaeota;D_2__Halobacteria;D_3__Halobacteriales;D_4__Halococcaceae;__
1                                                0.0                                           
2                                                0.0                                           
3                                                0.0                                           


https://towardsdatascience.com/a-beginners-tutorial-on-building-an-ai-image-classifier-using-pytorch-6f85cb69cba7

In [45]:
#3 Now, we get the features and metadata

expr_df = microbe_df.iloc[:,1:1177]
phenotype_df_neat = microbe_df.iloc[:,1178:1184]
phenotype_df = phenotype_df_neat

#We can set 'sample_title' as index by:
phenotype_df.set_index('sample_title')

#Let's covert UC and CD to IBD
#df.loc[df['column'] == 'column_value', 'column'] = 'new_column_value' or df.loc[(df.column == 'column_value'), 'column'] = 'new_column_value'

#phenotype_df.loc[phenotype_df['uc_cd'] == 'CD', 'uc_cd'] = 'IBD'
#phenotype_df.loc[phenotype_df['uc_cd'] == 'UC', 'uc_cd'] = 'IBD'

phenotype_df.loc[(phenotype_df.uc_cd == 'CD'), 'uc_cd'] = 'IBD'
phenotype_df.loc[(phenotype_df.uc_cd == 'UC'), 'uc_cd'] = 'IBD'

In [46]:
phenotype_df

,col_site,diagnosis,sample_title,stool_biopsy,studyID,uc_cd
0,OSCCAR,1,1939.100001,stool,GEVERSM,IBD
1,OSCCAR,1,1939.100003,stool,GEVERSM,IBD
2,OSCCAR,1,1939.100009,stool,GEVERSM,IBD
3,OSCCAR,1,1939.100015,stool,GEVERSM,IBD
4,OSCCAR,1,1939.100016,stool,GEVERSM,IBD
...,...,...,...,...,...,...
5262,AG,0,10317.000001870,stool,AG,Control
5263,AG,0,10317.000013111,stool,AG,Control
5264,AG,0,10317.000010960,stool,AG,Control
5265,AG,0,10317.000013005,stool,AG,Control


In [48]:
class MicrobDataset(Dataset):
  """
  Dataset for binary classification Tumor/Normal
  """
  def __init__(self):
    
    # Select rows whose type is Tumor or Normal
    self.labels = phenotype_df[phenotype_df["uc_cd"].apply(lambda s: s == "IBD" or s == "Control")]

    # Compute categorical embedding, 0 is Normal, 1 is Tumor
    self.labels = self.labels["uc_cd"].apply(lambda s: s == "Control").astype(int)

    # Get corresponding gene expression profiles
    self.X = expr_df

  def __getitem__(self, index):
    sample = np.array(self.X.iloc[index], dtype=np.float32)
    label = np.array(self.labels.iloc[index], dtype=np.float32)

    return sample, label

  def __len__(self):
    return len(self.labels)

In [49]:
dataset = MicrobDataset()
num_examples, num_genes = dataset.X.shape

In [51]:
print("Dataset for IBD/Control classification created with", num_examples, 
      "number of samples. Each sample contains the expression levels of", num_genes, "OTUs.")

Dataset for IBD/Control classification created with 5267 number of samples. Each sample contains the expression levels of 1176 OTUs.


In [52]:
train_set_size = int(len(dataset) * 0.7)
test_set_size = len(dataset) - train_set_size

In [53]:
train_dataset, test_dataset = torch.utils.data.random_split(dataset, 
                                                            lengths=[train_set_size, test_set_size], 
                                                            generator=torch.Generator().manual_seed(0))

In [54]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True, num_workers=2)

In [55]:
class LogisticRegression(nn.Module):
  def __init__(self, input_dim):
      super(LogisticRegression, self).__init__()

      # Initialize linear layer ()
      self.linear = nn.Linear(input_dim, 1)

  def forward(self, x):
    x = self.linear(x)  # Compute beta . x
    # We do not apply the sigmoid here, as it will be applied within the criterion function (for numerical stability)
    return x

In [56]:
model = LogisticRegression(num_genes)

In [59]:
optimizer = optim.Adam(model.parameters(), lr=0.00005)

In [60]:
criterion = nn.BCEWithLogitsLoss()

In [61]:
def compute_accuracy(loader, net):
  correct = 0
  total = 0
  with torch.no_grad():
      for data in loader:
          inputs, labels = data
          outputs = net(inputs)
          predicted = (outputs > 0).int().T

          total += labels.size(0)
          correct += (predicted == labels).sum().item()

  return 100 * correct / total

In [65]:
for epoch in range(100):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs[:, 0], labels)
        loss.backward()
        optimizer.step()  # Update the parameters of the model

        # print statistics
        running_loss += loss.item()
        if i % 50 == 49:    # print every 50 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 50))
            running_loss = 0.0

print('Finished Training')

[1,    50] loss: 33.903
[1,   100] loss: 29.338
[2,    50] loss: 29.923
[2,   100] loss: 21.507
[3,    50] loss: 22.700
[3,   100] loss: 18.701
[4,    50] loss: 17.327
[4,   100] loss: 14.619
[5,    50] loss: 12.868
[5,   100] loss: 13.020
[6,    50] loss: 12.581
[6,   100] loss: 10.162
[7,    50] loss: 11.532
[7,   100] loss: 8.143
[8,    50] loss: 8.541
[8,   100] loss: 8.885
[9,    50] loss: 8.619
[9,   100] loss: 7.405
[10,    50] loss: 8.162
[10,   100] loss: 6.145
[11,    50] loss: 6.112
[11,   100] loss: 7.040
[12,    50] loss: 6.971
[12,   100] loss: 4.778
[13,    50] loss: 5.731
[13,   100] loss: 5.352
[14,    50] loss: 4.365
[14,   100] loss: 5.289
[15,    50] loss: 4.314
[15,   100] loss: 4.223
[16,    50] loss: 3.301
[16,   100] loss: 4.435
[17,    50] loss: 3.648
[17,   100] loss: 3.091
[18,    50] loss: 3.122
[18,   100] loss: 3.563
[19,    50] loss: 3.128
[19,   100] loss: 2.607
[20,    50] loss: 2.639
[20,   100] loss: 2.818
[21,    50] loss: 2.658
[21,   100] loss: 2.4

In [ ]:
trained_model_file = F'/content/gdrive/MyDrive/PhD/Coursework/AI in Genomics/Projects/Phyla/phyla_dataset_d3/trained_model.pth'
torch.save(model.state_dict(), trained_model_file)

In [ ]:
model = LogisticRegression(num_genes)
model.load_state_dict(torch.load(trained_model_file))

In [66]:
print('Accuracy of the network on the ' + str(len(train_dataset)) + ' train samples: %d %%' % compute_accuracy(train_loader, model))

Accuracy of the network on the 3686 train samples: 92 %


In [67]:
print('Accuracy of the network on the ' + str(len(test_dataset)) + ' test samples: %d %%' % compute_accuracy(test_loader, model))

Accuracy of the network on the 1581 test samples: 88 %
